In [19]:
import numpy as np

In [41]:
class Quaternion(np.ndarray):
    def __new__(cls, input_vec=np.ones(4)):
        obj = np.asarray(input_vec).view(cls)
        if obj.shape[-1] == 3:
            obj = np.concatenate(((0,), obj))
        if obj.shape[-1] != 4:
            msg = 'Quaternions can only be created from 3- or 4-vectors.'
            raise ValueError(msg)
        return obj
    
    def __array_finalize__(self, obj):
        if obj is None:
            return
    
    def __add__(self, vec_):
        """
        Quaternion addition between two quaternions or a quaternion and a
        3-vector.
        """
        lhs = self.view(np.ndarray)
        rhs = np.asarray(vec_)
        return Quaternion(lhs + rhs)
    
    def __radd__(self, vec_):
        return self + vec_
            
    def __mul__(self, vec_):
        """
        Quaternion multiplication between two quaternions or a quaternion and a
        3-vector.
        
        $$
            ab = (a_0 b_0 - \vec{a}\cdot\vec{b}; a_0 \vec{b} + b_0 \vec{a} + \vec{a} \times \vec{b})
        $$
        """
        vec_ = Quaternion(vec_)
        a1, b1, c1, d1 = self
        a2, b2, c2, d2 = vec_
        result = Quaternion([
                a1*a2 - b1*b2 - c1*c2 - d1*d2,
                a1*b2 + b1*a2 + c1*d2 - d1*c2,
                a1*c2 - b1*d2 + c1*a2 + d1*b2,
                a1*d2 + b1*c2 - c1*b2 + d1*a2])
        return result

In [102]:
# implementations from https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation
def asquaternion(vec_):
    if isinstance(vec_, Quaternion):
        return vec_
    else:
        return Quaternion(vec_)
    
def normalize(quaternion):
    """
    Normalize the quaternion. A normalized quaternion is of the form
    
    $$
        \cos(\theta/2) + \sin(\theta/2) \vec{v}/|\vec{v}|
    $$
    """
    quaternion = asquaternion(quaternion)
    cq = quaternion[0]
    sq = np.sqrt(1. - cq**2)
    quaternion[1:4] /= np.linalg.norm(quaternion[1:4])
    quaternion[1:4] *= sq
    return quaternion
    
def inverse(quaternion):
    """
    Returns the inverse of the quaternion.
    """
    result  = np.copy(Quaternion(quaternion)).view(np.ndarray)
    result *= (1, -1, -1, -1)
    return asquaternion(result)
    
def from_matrix(matrix_):
    """
    Sets the quaternion from the rotation matrix.
    """
    if matrix_.shape != (3,3):
        raise ValueError('Rotation matrix must be a (3,3) matrix.')
    xx, xy, xz = matrix_[:,0]
    yx, yy, yz = matrix_[:,1]
    zx, zy, zz = matrix_[:,2]
    K = 1./3.*np.array([
            [xx - yy - zz, yx + xy, zx + xz, yz - zy],
            [yx + xy, yy - xx - zz, zy + yz, zx - xz],
            [zx + xz, zy + yz, zz - xx - yy, xy - yx],
            [yz - zy, zx - xz, xy - yx, xx + yy + zz]])
    # the constructs a quaternion that is a closest fit to the
    # rotation in `matrix_`. If `matrix_` is pure rotation,
    # then `s[0]` is 1.
    U,s,V = np.linalg.svd(K, full_matrices=False)
    if not np.isclose(s[0], 1):
        raise ValueError('Rotation matrix {} is not a pure rotation.'.format(matrix_))
    (x, y, z, w) = U[:,0]
    return Quaternion((w, x, y, z))

def from_axis_angle(axis_, angle_):
    """
    Sets the quaternion from the axis/angle pair. Angle should be in degrees.
    """
    axis_ = np.asarray(axis_)/np.linalg.norm(axis_)
    # Note: quaternion uses half angles because rotation w' <- q w q*
    angle_ = np.radians(angle_)/2.
    cq = np.cos(angle_)
    sq = np.sin(angle_)
    return Quaternion(np.concatenate(((cq,), sq*axis_)))
    
def from_u_to_v(u_, v_):
    """
    Sets the quaternion to rotation `u` into `v`.
    """
    u_ = np.copy(u_)/np.linalg.norm(u_)
    v_ = np.copy(v_)/np.linalg.norm(v_)
    axis_ = np.cross(u_, v_)
    angle_ = np.degrees(np.arccos(np.dot(u_, v_)))
    return from_axis_angle(axis_, angle_)
    
def to_matrix(quaternion):
    """
    Converts the quaternion into an equivalent rotation matrix.
    """
    quaternion = asquaternion(quaternion)
    a, b, c, d = normalize(quaternion)
    return np.array([
            [a*a + b*b - c*c - d*d, 2*b*c - 2*a*d, 2*b*d + 2*a*c],
            [2*b*c + 2*a*d, a*a - b*b + c*c - d*d, 2*c*d - 2*a*b],
            [2*b*d - 2*a*c, 2*c*d + 2*a*b, a*a - b*b - c*c + d*d]])

def to_axis_angle(quaternion):
    """
    Converts the quaternion into an equivalent axis/angle pair.
    The angle is given in degrees. The vector is normalized.
    """
    angle_ = np.degrees(2.*np.arccos(quaternion[0]))
    axis_  = np.asarray(quaternion[1:4])
    axis_ /= np.linalg.norm(axis_)
    return (axis_, angle_)

## Test quaterion functions

In [112]:
Rz = lambda q: np.array([
        [ np.cos(q),  np.sin(q), 0],
        [-np.sin(q),  np.cos(q), 0],
        [         0,          0, 1]])

Rx = lambda q: np.array([
        [ 1,          0,          0],
        [ 0,  np.cos(q),  np.sin(q)],
        [ 0, -np.sin(q),  np.cos(q)]])

def euler_matrix(phi, theta, psi):
    """
    All angles are in degrees.
    
    1. Rotation by angle phi about the z-axis
    2. Rotation by angle theta about the former x-axis (now x')
    3. Rotation by angle psi about the former z-axis (now z')
    """
    assert (0 <= theta <= 180.), \
        "Theta must lie between 0 and 180 degrees, inclusively."
    phi = np.radians(phi)
    theta = np.radians(theta)
    psi = np.radians(psi)
    return np.dot(Rz(psi), np.dot(Rx(theta), Rz(phi)))

In [128]:
# check conversion from rotation matrix
theta, phi, psi = 180*np.random.random(3)

# create test vectors
x = np.random.random(3)
x /= np.linalg.norm(x)

u,v = np.random.random((2, 3))
u /= np.linalg.norm(u)
v /= np.linalg.norm(v)

axis, angle = np.random.random(3), 180*np.random.random()
axis /= np.linalg.norm(axis)

# create matrices
R = euler_matrix(theta, phi, psi)
Qm = from_matrix(R)
Quv = from_u_to_v(u, v)
Qaa = from_axis_angle(axis, angle)

In [129]:
# test matrix rotation
rxR = np.dot(R, x[:, np.newaxis]).flatten()
# quaternion rotation
rxQ = Qm*x*inverse(Qm)

# verify
print 'Quaternion rotation matches matrix rotation... ',
if np.allclose(rxR, np.asarray(rxQ[1:])):
    print "pass"
else:
    print "fail"

Quaternion rotation matches matrix rotation...  pass


In [130]:
# check conversion to rotation matrix
print "Quaternion conversion to matrix...",
if np.allclose(to_matrix(Qm), R):
    print "pass"
else:
    print "fail"

Quaternion conversion to matrix... pass


In [ ]:
# TODO: test axis/angle rotation
# create a rotation matrix from an axis/angle pair
# perform axis/angle rotation
# perform quaternion rotation
# compare axis/angle rotation to equivalent quaternion rotation

In [131]:
# check conversion to axis/angle
print "Quaternion conversion to axis/angle...",
caxis, cangle = to_axis_angle(Qaa)
if np.isclose(cangle, angle) and np.allclose(caxis, axis):
    print "pass"
else:
    print "fail: ({}, {}) != ({}, {})".format(caxis, cangle, axis, angle)

Quaternion conversion to axis/angle... pass
